In [26]:
import json
import os,glob

In [27]:
folder_path = 'D:\\COVID-19-TweetIDs\\2020-01\\New folder\\22'

In [28]:
location = []
for filename in glob.glob(os.path.join(folder_path, '*.jsonl')):
    fhand = open(filename, 'r',encoding="utf8")
    for line in fhand:
        tweet = json.loads(line)
        try:
            location.append(tweet['user']['location'])
        except:
            continue

In [29]:
import geonamescache
gc = geonamescache.GeonamesCache()

In [30]:
#create a list geolocation of 180 elements type(dict) in the form city_code:[list of info]
user_number = 0
geolocation= []
for i in location:
    cities = i.split(",")[0].title() #use.title to make cities like hong kong to Hong Kong
    user_number+=1
    ct = gc.get_cities_by_name(cities)
    if(ct!= []):
        geolocation.append(ct[0]) #some get_cities_by_name returns 2 or 3 cities so we just get the first result

In [31]:
#create a citystatecountry that is a list of 180 locations, each has 3 elements: name, admin1code(state), country.
citystatecountry=[]
#create a listkey of citycodes len = 180
listkey=[]
for i in geolocation:
    key = list(i.keys())[0]
    listkey.append(key)
len(listkey)#this is the list with repeated codes.

180

In [32]:
#count tweet using count, and get rid of repeated codes
#create the dictionary citycode: number_of_tweet. citycode = different citycodes. number_of_tweet = count. len=122
tweet_count = {i:listkey.count(i) for i in listkey}
print(len(tweet_count))

122


In [33]:
#write lists of location
abc = []#abc get rid of repeated locations from the geolocation
for i in geolocation:
    if(i not in abc):
        abc.append(i)
#print(abc)

In [34]:
#create different lists of city, states and country based on abc
keylist = []
a = []
city = []
state = []
country = []
for i in abc:
    key = list(i.keys())[0]#the geonamescache returns several results for every input, so we get just the first result.
    keylist.append(key)
    infocity = i.get(key)
    name = infocity.get('name')
    city.append(name)
    state1 = infocity.get('admin1code')
    state.append(state1)
    country1 = infocity.get('countrycode')
    country.append(country1)

In [35]:
#create a csv file based on dictionary tweet_count
import csv
with open('test.csv', 'w') as f:
    for key in tweet_count.keys():
        f.write("%s,%s\n"%(key,tweet_count[key]))

In [36]:
#create a dictionary. key = citycode, value = lists of location. Zip them up
new_dict = {i: [j,k,l] for i,j,k,l in zip(keylist,city,country,state)}

In [37]:
#write the dictionary above to a csv file 
with open('location.csv', 'w') as f:
    for key in new_dict.keys():
        f.write("%s,%s\n"%(key,new_dict[key]))

In [40]:
#merge the two csv files based on the key using pandas
import pandas as pd
a = pd.read_csv("location.csv", encoding = "ISO-8859-1") #utf8 encoding problem so I have to add ISO
b = pd.read_csv("test.csv", encoding = "ISO-8859-1")
b = b.dropna(axis=1)
merged = a.merge(b)
merged.to_csv("test1.csv", index=False)

In [41]:
#edit the file to add headings and get rid of 
s = open('test1.csv','r').read()
chars = ('[',']') # etc
for c in chars:
  s = ''.join( s.split(c) )

out_file = open('test2.csv','w')
fields= u'city_code,city,country,state,tweet_count'
out_file.write(fields)
out_file.write(u'\n')
out_file.write(s)
out_file.close()

In [47]:
#read the result file
with open('test2.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in reader:
        print(', '.join(row))

city_code,city,country,state,tweet_count
2634715,'Washington',, 'GB',, 'ENG',5
4180439,'Atlanta',, 'US',, 'GA',6
5380184,'Oxnard',, 'US',, 'CA',1
4364946,'Parkville',, 'US',, 'MD',1
4887398,'Chicago',, 'US',, 'IL',7
5037649,'Minneapolis',, 'US',, 'MN',1
2641598,'Newport',, 'GB',, 'WLS',1
5574991,'Boulder',, 'US',, 'CO',1
2650628,'Durham',, 'GB',, 'ENG',1
6183235,'Winnipeg',, 'CA',, '03',2
4012176,'CuliacÃ¡n',, 'MX',, '25',1
2643123,'Manchester',, 'GB',, 'ENG',1
6167865,'Toronto',, 'CA',, '08',4
4633419,'Johnson, City',, 'US',, 'TN',1
4720131,'Portland',, 'US',, 'TX',1
2655603,'Birmingham',, 'GB',, 'ENG',1
1850147,'Tokyo',, 'JP',, '40',1
4641239,'Memphis',, 'US',, 'TN',2
4259418,'Indianapolis',, 'US',, 'IN',1
5379439,'Ontario',, 'US',, 'CA',5
633679,'Turku',, 'FI',, '02',1
729114,'Montana',, 'BG',, '47',1
3654667,'Loja',, 'EC',, '12',1
943882,'Virginia',, 'ZA',, '03',1
3682393,'Florida',, 'CO',, '29',4
3675707,'Madrid',, 'CO',, '33',1
3429732,'Puerto, Rico',, 'AR',, '14',2
3471872,'Arac